# Customizing embeddings

This notebook demonstrates one way to customize OpenAI embeddings to a particular task.

The input is training data in the form of [text_1, text_2, label] where label is +1 if the pairs are similar and -1 if the pairs are dissimilar.

The output is a matrix that you can use to multiply your embeddings. The product of this multiplication is a 'custom embedding' that will better emphasize aspects of the text relevant to your use case. In binary classification use cases, we've seen error rates drop by as much as 50%.

In the following example, I use 1,000 sentence pairs picked from the SNLI corpus. Each pair of sentences are logically entailed (i.e., one implies the other). These pairs are our positives (label = 1). We generate synthetic negatives by combining sentences from different pairs, which are presumed to not be logically entailed (label = -1).

For a clustering use case, you can generate positives by creating pairs from texts in the same clusters and generate negatives by creating pairs from sentences in different clusters.

With other data sets, we have seen decent improvement with as little as ~100 training examples. Of course, performance will be better with  more examples.

# 0. Imports

In [347]:
# imports
from typing import List, Tuple  # for type hints
import numpy as np  # for manipulating arrays
import pandas as pd  # for manipulating data in dataframes
import pickle  # for saving the embeddings cache
import plotly.express as px  # for plots
import random  # for generating run IDs
from sklearn.model_selection import train_test_split  # for splitting train & test data
import torch  # for matrix optimization

from utils.embeddings_utils import get_embedding, cosine_similarity  # for embeddings


## 1. Inputs

Most inputs are here. The key things to change are where to load your datset from, where to save a cache of embeddings to, and which embedding engine you want to use.

Depending on how your data is formatted, you'll want to rewrite the process_input_data function.

In [348]:
# input parameters
embedding_cache_path = "data/snli_embedding_cache.pkl"  # embeddings will be saved/loaded here
default_embedding_engine = "text-embedding-3-small"
num_pairs_to_embed = 1000  # 1000 is arbitrary
local_dataset_path = "data/snli_1.0_train_2k.csv"  # download from: https://nlp.stanford.edu/projects/snli/


def process_input_data(df: pd.DataFrame) -> pd.DataFrame:
    # you can customize this to preprocess your own dataset
    # output should be a dataframe with 3 columns: text_1, text_2, label (1 for similar, -1 for dissimilar)
    df["label"] = df["gold_label"]
    df = df[df["label"].isin(["entailment"])]
    df.loc[:, "label"] = df["label"].apply(lambda x: {"entailment": 1, "contradiction": -1}[x])
    df = df.rename(columns={"sentence1": "text_1", "sentence2": "text_2"})
    df = df[["text_1", "text_2", "label"]]
    df = df.head(num_pairs_to_embed)
    return df


## 2. Load and process input data

In [349]:
# load data
df = pd.read_csv(local_dataset_path)

# process input data
df = process_input_data(df)  # this demonstrates training data containing only positives

# view data
print(len(df))
df.head()


669


,text_1,text_2,label
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1
4,Children smiling and waving at camera,There are children present,1
7,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,1
14,Two blond women are hugging one another.,There are women showing affection.,1
17,"A few people in a restaurant setting, one of t...",The diners are at a restaurant.,1


## 3. Split data into training test sets

Note that it's important to split data into training and test sets *before* generating synethetic negatives or positives. You don't want any text strings in the training data to show up in the test data. If there's contamination, the test metrics will look better than they'll actually be in production.

In [350]:
# split data into train and test sets
# todo for test
test_fraction = 0.1  # 0.5 is fairly arbitrary
# todo debug, generaly, train_size is not nessary
train_size = 0.1
random_seed = 123  # random seed is arbitrary, but is helpful in reproducibility
train_df, test_df = train_test_split(
    df, test_size=test_fraction, train_size=train_size, stratify=df["label"], random_state=random_seed
)
train_df.loc[:, "dataset"] = "train"
test_df.loc[:, "dataset"] = "test"
print(len(train_df), len(test_df), )
print(train_df.head())
print(test_df.head())


66 67
                                                 text_1  \
1700  There are four Eastern guys working on and han...   
1965  The surfer catches a big wave but stays on his...   
1432  Two women, each with a child, look at each other.   
1686  Five men, one wearing a white shirt standing o...   
931   People in a meeting setting paying attention t...   

                                               text_2 label dataset  
1700                             The men are working.     1   train  
1965                           The person is surfing.     1   train  
1432                        Women are with their kids     1   train  
1686  A group of men are hanging a picture on a wall.     1   train  
931            The people are listening to a speaker.     1   train  
                                                 text_1  \
462   A man stare at a passing couple while walking ...   
1106      Young woman in a cafe checking her cellphone.   
31    A Little League team tries to catch 

## 4. Generate synthetic negatives

This is another piece of the code that you will need to modify to match your use case.

If you have data with positives and negatives, you can skip this section.

If you have data with only positives, you can mostly keep it as is, where it generates negatives only.

If you have multiclass data, you will want to generate both positives and negatives. The positives can be pairs of text that share labels, and the negatives can be pairs of text that do not share labels.

The final output should be a dataframe with text pairs, where each pair is labeled -1 or 1.

In [383]:
# generate negatives
def dataframe_of_negatives(dataframe_of_positives: pd.DataFrame) -> pd.DataFrame:
    """Return dataframe of negative pairs made by combining elements of positive pairs."""
    texts = set(dataframe_of_positives["text_1"].values) | set(
        dataframe_of_positives["text_2"].values
    )
    all_pairs = {(t1, t2) for t1 in texts for t2 in texts if t1 < t2}
    positive_pairs = set(
        tuple(text_pair)
        for text_pair in dataframe_of_positives[["text_1", "text_2"]].values
    )
    negative_pairs = all_pairs - positive_pairs
    df_of_negatives = pd.DataFrame(list(negative_pairs), columns=["text_1", "text_2"])
    df_of_negatives["label"] = -1
    return df_of_negatives


{('text2', 'text3'), ('text1', 'text2'), ('text1', 'text3'), ('text3', 'text4'), ('text3', 'text5'), ('text4', 'text5'), ('text2', 'text5'), ('text1', 'text4'), ('text1', 'text5'), ('text2', 'text4')}


In [352]:
negatives_per_positive = (
    1  # it will work at higher values too, but more data will be slower
)
# generate negatives for training dataset
train_df_negatives = dataframe_of_negatives(train_df)
train_df_negatives["dataset"] = "train"
# generate negatives for test dataset
test_df_negatives = dataframe_of_negatives(test_df)
test_df_negatives["dataset"] = "test"
# sample negatives and combine with positives
train_df = pd.concat(
    [
        train_df,
        train_df_negatives.sample(
            n=len(train_df) * negatives_per_positive, random_state=random_seed
        ),
    ]
)
test_df = pd.concat(
    [
        test_df,
        test_df_negatives.sample(
            n=len(test_df) * negatives_per_positive, random_state=random_seed
        ),
    ]
)

df = pd.concat([train_df, test_df])
print(len(df))
df.head()


266


,text_1,text_2,label,dataset
1700,There are four Eastern guys working on and han...,The men are working.,1,train
1965,The surfer catches a big wave but stays on his...,The person is surfing.,1,train
1432,"Two women, each with a child, look at each other.",Women are with their kids,1,train
1686,"Five men, one wearing a white shirt standing o...",A group of men are hanging a picture on a wall.,1,train
931,People in a meeting setting paying attention t...,The people are listening to a speaker.,1,train


## 5. Calculate embeddings and cosine similarities

Here, I create a cache to save the embeddings. This is handy so that you don't have to pay again if you want to run the code again.

In [353]:
# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, engine) -> embedding
try:
    with open(embedding_cache_path, "rb") as f:
        embedding_cache = pickle.load(f)
except FileNotFoundError:
    precomputed_embedding_cache_path = "https://cdn.openai.com/API/examples/data/snli_embedding_cache.pkl"
    # todo cache file is too large
    # embedding_cache = pd.read_pickle(precomputed_embedding_cache_path)
    embedding_cache = pd.DataFrame()

df = df.dropna()

# this function will get embeddings from the cache and save them there afterward
def get_embedding_with_cache(
    text: str,
    engine: str = default_embedding_engine,
    embedding_cache: dict = embedding_cache,
) -> list:
    if (text, engine) not in embedding_cache.keys():
        # if not in cache, call API to get embedding
        embedding_cache[(text, engine)] = get_embedding(text, engine)
    return embedding_cache[(text, engine)]

# create column of embeddings
for column in ["text_1", "text_2"]:
    df_embedding = df.apply(lambda row: get_embedding_with_cache(row[column]).tolist(), axis=1)
    df.loc[:, f"{column}_embedding"] = df_embedding

# create column of cosine similarity between embeddings
df["cosine_similarity"] = df.apply(
    lambda row: cosine_similarity(row["text_1_embedding"], row["text_2_embedding"]),
    axis=1,
)
# save embeddings cache to disk after each update
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

df.head()


get_embedding:  Usage(prompt_tokens=13, total_tokens=13)
get_embedding:  Usage(prompt_tokens=20, total_tokens=20)


get_embedding:  Usage(prompt_tokens=15, total_tokens=15)
get_embedding:  Usage(prompt_tokens=7, total_tokens=7)
get_embedding:  Usage(prompt_tokens=15, total_tokens=15)
get_embedding:  Usage(prompt_tokens=12, total_tokens=12)
get_embedding:  Usage(prompt_tokens=17, total_tokens=17)
get_embedding:  Usage(prompt_tokens=23, total_tokens=23)
get_embedding:  Usage(prompt_tokens=17, total_tokens=17)
get_embedding:  Usage(prompt_tokens=14, total_tokens=14)
get_embedding:  Usage(prompt_tokens=18, total_tokens=18)
get_embedding:  Usage(prompt_tokens=21, total_tokens=21)
get_embedding:  Usage(prompt_tokens=12, total_tokens=12)
get_embedding:  Usage(prompt_tokens=15, total_tokens=15)
get_embedding:  Usage(prompt_tokens=20, total_tokens=20)
get_embedding:  Usage(prompt_tokens=28, total_tokens=28)
get_embedding:  Usage(prompt_tokens=21, total_tokens=21)
get_embedding:  Usage(prompt_tokens=8, total_tokens=8)
get_embedding:  Usage(prompt_tokens=22, total_tokens=22)
get_embedding:  Usage(prompt_tokens

/tmp/ipykernel_2250/3149054672.py:22: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



get_embedding:  Usage(prompt_tokens=33, total_tokens=33)


/tmp/ipykernel_2250/3149054672.py:22: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



...

get_embedding:  Usage(prompt_tokens=7, total_tokens=7)


/tmp/ipykernel_2250/3149054672.py:22: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



get_embedding:  Usage(prompt_tokens=4, total_tokens=4)


/tmp/ipykernel_2250/3149054672.py:22: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



,text_1,text_2,label,dataset,text_1_embedding,text_2_embedding,cosine_similarity
1700,There are four Eastern guys working on and han...,The men are working.,1,train,"[-0.0234934464097023, 0.06062634661793709, 0.0...","[-0.014497929252684116, 0.06136217713356018, -...",0.422816
1965,The surfer catches a big wave but stays on his...,The person is surfing.,1,train,"[0.04675769805908203, 0.04438018798828125, -0....","[0.03581292927265167, 0.022507816553115845, -0...",0.528475
1432,"Two women, each with a child, look at each other.",Women are with their kids,1,train,"[0.02762751840054989, 0.004032856784760952, -0...","[0.04005732014775276, 0.04540528729557991, 0.0...",0.576967
1686,"Five men, one wearing a white shirt standing o...",A group of men are hanging a picture on a wall.,1,train,"[0.00759082892909646, 0.03502237796783447, -0....","[-0.0018928300123661757, 0.020557211712002754,...",0.589328
931,People in a meeting setting paying attention t...,The people are listening to a speaker.,1,train,"[-0.013276001438498497, 0.019361671060323715, ...","[0.021909121423959732, -0.016262710094451904, ...",0.601007


## 6. Plot distribution of cosine similarity

Here we measure similarity of text using cosine similarity. In our experience, most distance functions (L1, L2, cosine similarity) all work about the same. Note that our embeddings are already normalized to length 1, so cosine similarity is equivalent to dot product.

The graphs show how much the overlap there is between the distribution of cosine similarities for similar and dissimilar pairs. If there is a high amount of overlap, that means there are some dissimilar pairs with greater cosine similarity than some similar pairs.

The accuracy I compute is the accuracy of a simple rule that predicts 'similar (1)' if the cosine similarity is above some threshold X and otherwise predicts 'dissimilar (0)'.

In [354]:
# calculate accuracy (and its standard error) of predicting label=1 if similarity>x
# x is optimized by sweeping from -1 to 1 in steps of 0.01
def accuracy_and_se(cosine_similarity: float, labeled_similarity: int) -> Tuple[float]:
    accuracies = []
    for threshold_thousandths in range(-1000, 1000, 1):
        threshold = threshold_thousandths / 1000
        total = 0
        correct = 0
        for cs, ls in zip(cosine_similarity, labeled_similarity):
            total += 1
            if cs > threshold:
                prediction = 1
            else:
                prediction = -1
            if prediction == ls:
                correct += 1
        accuracy = correct / total
        accuracies.append(accuracy)
    a = max(accuracies)
    n = len(cosine_similarity)
    standard_error = (a * (1 - a) / n) ** 0.5  # standard error of binomial
    return a, standard_error


# check that training and test sets are balanced
px.histogram(
    df,
    x="cosine_similarity",
    color="label",
    barmode="overlay",
    width=500,
    facet_row="dataset",
).show()

for dataset in ["train", "test"]:
    data = df[df["dataset"] == dataset]
    a, se = accuracy_and_se(data["cosine_similarity"], data["label"])
    print(f"{dataset} accuracy: {a:0.1%} ± {1.96 * se:0.1%}")


train accuracy: 97.0% ± 2.9%
test accuracy: 97.8% ± 2.5%


## 7. Optimize the matrix using the training data provided

In [355]:
def embedding_multiplied_by_matrix(
    embedding: List[float], matrix: torch.tensor
) -> np.array:
    embedding_tensor = torch.tensor(embedding).float()
    modified_embedding = embedding_tensor @ matrix
    modified_embedding = modified_embedding.detach().numpy()
    return modified_embedding


# compute custom embeddings and new cosine similarities
def apply_matrix_to_embeddings_dataframe(matrix: torch.tensor, df: pd.DataFrame):
    for column in ["text_1_embedding", "text_2_embedding"]:
        df[f"{column}_custom"] = df[column].apply(
            lambda x: embedding_multiplied_by_matrix(x, matrix)
        )
    df["cosine_similarity_custom"] = df.apply(
        lambda row: cosine_similarity(
            row["text_1_embedding_custom"], row["text_2_embedding_custom"]
        ),
        axis=1,
    )


In [380]:
def optimize_matrix(
    modified_embedding_length: int = 2048,  # in my brief experimentation, bigger was better (2048 is length of babbage encoding)
    batch_size: int = 100,
    max_epochs: int = 100,
    learning_rate: float = 100.0,  # seemed to work best when similar to batch size - feel free to try a range of values
    dropout_fraction: float = 0.0,  # in my testing, dropout helped by a couple percentage points (definitely not necessary)
    df: pd.DataFrame = df,
    print_progress: bool = True,
    save_results: bool = True,
) -> torch.tensor:
    """Return matrix optimized to minimize loss on training data."""
    run_id = random.randint(0, 2 ** 31 - 1)  # (range is arbitrary)
    # convert from dataframe to torch tensors
    # e is for embedding, s for similarity label
    def tensors_from_dataframe(
        df: pd.DataFrame,
        embedding_column_1: str,
        embedding_column_2: str,
        similarity_label_column: str,
    ) -> Tuple[torch.tensor]:
        e1 = np.stack(np.array(df[embedding_column_1].values))
        e2 = np.stack(np.array(df[embedding_column_2].values))
        s = np.stack(np.array(df[similarity_label_column].astype("float").values))

        e1 = torch.from_numpy(e1).float()
        e2 = torch.from_numpy(e2).float()
        s = torch.from_numpy(s).float()

        return e1, e2, s

    e1_train, e2_train, s_train = tensors_from_dataframe(
        df[df["dataset"] == "train"], "text_1_embedding", "text_2_embedding", "label"
    )
    e1_test, e2_test, s_test = tensors_from_dataframe(
        df[df["dataset"] == "test"], "text_1_embedding", "text_2_embedding", "label"
    )

    # create dataset and loader
    dataset = torch.utils.data.TensorDataset(e1_train, e2_train, s_train)
    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=True
    )

    # define model (similarity of projected embeddings)
    def model(embedding_1, embedding_2, matrix, dropout_fraction=dropout_fraction):
        e1 = torch.nn.functional.dropout(embedding_1, p=dropout_fraction)
        e2 = torch.nn.functional.dropout(embedding_2, p=dropout_fraction)
        modified_embedding_1 = e1 @ matrix  # @ is matrix multiplication
        modified_embedding_2 = e2 @ matrix
        similarity = torch.nn.functional.cosine_similarity(
            modified_embedding_1, modified_embedding_2
        )
        return similarity

    # define loss function to minimize
    def mse_loss(predictions, targets):
        difference = predictions - targets
        return torch.sum(difference * difference) / difference.numel()

    # initialize projection matrix
    embedding_length = len(df["text_1_embedding"].values[0])
    matrix = torch.randn(
        embedding_length, modified_embedding_length, requires_grad=True
    )

    epochs, types, losses, accuracies, matrices = [], [], [], [], []
    for epoch in range(1, 1 + max_epochs):
        # iterate through training dataloader
        for a, b, actual_similarity in train_loader:
            # generate prediction
            predicted_similarity = model(a, b, matrix)
            # get loss and perform backpropagation
            loss = mse_loss(predicted_similarity, actual_similarity)
            loss.backward()
            # update the weights
            with torch.no_grad():
                matrix -= matrix.grad * learning_rate
                # set gradients to zero
                matrix.grad.zero_()
        # calculate test loss
        test_predictions = model(e1_test, e2_test, matrix)
        test_loss = mse_loss(test_predictions, s_test)

        # compute custom embeddings and new cosine similarities
        apply_matrix_to_embeddings_dataframe(matrix, df)

        # calculate test accuracy
        for dataset in ["train", "test"]:
            data = df[df["dataset"] == dataset]
            a, se = accuracy_and_se(data["cosine_similarity_custom"], data["label"])

            # record results of each epoch
            epochs.append(epoch)
            types.append(dataset)
            losses.append(loss.item() if dataset == "train" else test_loss.item())
            accuracies.append(a)
            matrices.append(matrix.detach().numpy())

            # optionally print accuracies
            if print_progress is True:
                print(
                    f"Epoch {epoch}/{max_epochs}: {dataset} accuracy: {a:0.1%} ± {1.96 * se:0.1%}"
                )

    data = pd.DataFrame(
        {"epoch": epochs, "type": types, "loss": losses, "accuracy": accuracies}
    )
    data["run_id"] = run_id
    data["modified_embedding_length"] = modified_embedding_length
    data["batch_size"] = batch_size
    data["max_epochs"] = max_epochs
    data["learning_rate"] = learning_rate
    data["dropout_fraction"] = dropout_fraction
    data[
        "matrix"
    ] = matrices  # saving every single matrix can get big; feel free to delete/change
    if save_results is True:
        data.to_csv(f"{run_id}_optimization_results.csv", index=False)

    return data


In [379]:
# example hyperparameter search
# I recommend starting with max_epochs=10 while initially exploring
results = []
# todo debug
max_epochs = 1 # 30
dropout_fraction = 0.2
for batch_size, learning_rate in [(10, 10), (100, 100), (1000, 1000)]:
    result = optimize_matrix(
        batch_size=batch_size,
        learning_rate=learning_rate,
        max_epochs=max_epochs,
        dropout_fraction=dropout_fraction,
        save_results=False,
    )
    results.append(result)


33333 tensor([0.1262, 0.4703, 0.4904, 0.0424, 0.0584, 0.5299, 0.4679, 0.0632, 0.1033,
        0.4881], grad_fn=<SumBackward1>)
33333 tensor([0.1204, 0.1820, 0.1410, 0.1631, 0.0254, 0.1647, 0.2129, 0.1529, 0.1926,
        0.1794], grad_fn=<SumBackward1>)
33333 tensor([0.4478, 0.6074, 0.0301, 0.4635, 0.5189, 0.4945, 0.7676, 0.1602, 0.0831,
        0.1760], grad_fn=<SumBackward1>)
33333 tensor([0.5507, 0.5850, 0.0999, 0.0138, 0.1822, 0.4425, 0.4747, 0.4725, 0.1536,
        0.6603], grad_fn=<SumBackward1>)
33333 tensor([ 0.1184,  0.1673, -0.0034,  0.1359,  0.5158,  0.3229,  0.1277,  0.6291,
         0.4414,  0.1707], grad_fn=<SumBackward1>)
33333 tensor([0.6703, 0.0964, 0.2367, 0.5052, 0.2895, 0.3104, 0.0729, 0.0409, 0.0907,
        0.4255], grad_fn=<SumBackward1>)
33333 tensor([0.5794, 0.7705, 0.1229, 0.3802, 0.3970, 0.2822, 0.5842, 0.2096, 0.1136,
        0.1360], grad_fn=<SumBackward1>)
33333 tensor([0.1660, 0.6279, 0.4395, 0.5474, 0.1633, 0.1331, 0.1189, 0.6150, 0.4661,
        0.1212]

In [358]:
runs_df = pd.concat(results)

# plot training loss and test loss over time
px.line(
    runs_df,
    line_group="run_id",
    x="epoch",
    y="loss",
    color="type",
    hover_data=["batch_size", "learning_rate", "dropout_fraction"],
    facet_row="learning_rate",
    facet_col="batch_size",
    width=500,
).show()

# plot accuracy over time
px.line(
    runs_df,
    line_group="run_id",
    x="epoch",
    y="accuracy",
    color="type",
    hover_data=["batch_size", "learning_rate", "dropout_fraction"],
    facet_row="learning_rate",
    facet_col="batch_size",
    width=500,
).show()


## 8. Plot the before & after, showing the results of the best matrix found during training

The better the matrix is, the more cleanly it will separate the similar and dissimilar pairs.

In [359]:
# apply result of best run to original data
best_run = runs_df.sort_values(by="accuracy", ascending=False).iloc[0]
best_matrix = best_run["matrix"]
apply_matrix_to_embeddings_dataframe(best_matrix, df)


In [360]:
# plot similarity distribution BEFORE customization
px.histogram(
    df,
    x="cosine_similarity",
    color="label",
    barmode="overlay",
    width=500,
    facet_row="dataset",
).show()

test_df = df[df["dataset"] == "test"]
a, se = accuracy_and_se(test_df["cosine_similarity"], test_df["label"])
print(f"Test accuracy: {a:0.1%} ± {1.96 * se:0.1%}")

# plot similarity distribution AFTER customization
px.histogram(
    df,
    x="cosine_similarity_custom",
    color="label",
    barmode="overlay",
    width=500,
    facet_row="dataset",
).show()

a, se = accuracy_and_se(test_df["cosine_similarity_custom"], test_df["label"])
print(f"Test accuracy after customization: {a:0.1%} ± {1.96 * se:0.1%}")


Test accuracy: 97.8% ± 2.5%


Test accuracy after customization: 97.8% ± 2.5%


In [361]:
best_matrix  # this is what you can multiply your embeddings by


array([[ 0.22042876,  0.87521535,  0.0420429 , ..., -0.09318765,
         0.8983297 ,  0.5960714 ],
       [ 1.2881154 , -0.85908407,  0.15188482, ..., -1.9789133 ,
        -0.04925788, -0.06381588],
       [-0.2158554 ,  0.922801  ,  1.3460056 , ..., -2.3042078 ,
        -2.6791139 , -0.368992  ],
       ...,
       [-0.5339413 , -1.4590217 ,  1.8016938 , ...,  1.1116925 ,
         0.25756225, -0.36961156],
       [ 1.3289257 ,  1.121087  ,  0.26818687, ..., -1.348178  ,
         0.830635  , -1.3762494 ],
       [ 0.84373903, -0.74059325, -0.6357079 , ...,  0.06219171,
         0.19869135, -0.37024224]], dtype=float32)